# PANDAS

> `pandas` is an open source, BSD-licensed library providing high-performance, easy-to-use data structures and data analysis tools for the Python programming language.

https://pandas.pydata.org/

Balík `pandas` je dnes prakticky standardem ve všech odvětvích, která mají co do činění s *data science*. Balík přináší do Pythonu koncep `DataFrame` objektů (jaký možná znáte např. z jazyka R), který usnadňuje práci s databázovými/tabulkovými data, přičemž prací se zde myslí vše od elementární statistiky až po komplexní filtrování a agregace společně s poctivou statistickou analýzou.

`pandas` je poměrně komplexní balík se spoustou funkcí, který se průběžně vyvíjí. V této kapitole proto naleznete jen základní přehled. Pro podrobnější studium doporučuji tutoriály přímo na webu projektu:

- https://pandas.pydata.org/docs/user_guide/10min.html
- https://pandas.pydata.org/docs/user_guide/cookbook.html


## Základní principy

Prvním stavebním kamenem je jednorozměrný objekt `pd.Series`. Je velmi podobný `numpy` polím - obsahuje hodnoty jednoho typu (libovolné python objekty) a je použitelný ve většině `numpy` funkcí. `pd.Series` má navíc k datům ještě značky (labels), obvykle označované jako __index__. Existuje nepřeberné množství způsobů, jak `Series` vytvářet.

In [ ]:
import pandas as pd

s = pd.Series(range(5), index=["a", "b", "c", "d", "e"])
print(s)

In [ ]:
d = { l:i for i, l in enumerate("abcdefghijklmnopqrstuvwxyz", start=1)}
s = pd.Series(d)
s.head()

`pandas` objekty podporují různé způsoby přístupu k datům. Klasická hranatá závorka je rezervovaná pro labels, běžné poziční indexování a slices fungují přes všudypřítomnou metodu `.iloc`.

In [ ]:
print(s["b"])
print(s["x":])
print(s.iloc[:2])
print(s.iloc[[0, 13, -1]])

Druhým a patrně nejčastěji užívaným kamenem je dvojrozměrný objekt `pd.DataFrame`. `DataFrame` je obecně tvořen sloupci obecně různých typů. Je užitečné na `DataFrame` nahlížet jako na slovník objektů `pd.Series`.

In [ ]:
d = {"one": [1.0, 2.0, 3.0, 4.0], "two": [4, 3, 2, 1], "three": ["eins", "zwei", "drei", "vier"]}
df = pd.DataFrame(d, index=["a", "b", "c", "d"])
df

In [ ]:
df["one"]
df.loc[["a", "c"]]
s = df.loc["a"]
print(s, type(s), s.dtype)

In [ ]:
s = df[["one", "two"]].loc["a"] # prekvapiva nevyzadana konverze
print(s, type(s), s.dtype)

`DataFrame` poskytuje i metodu postihující nějakou základní statistiku.

In [ ]:
df.describe()

Vybrané atributy objektu `DataFrame`

In [ ]:
df.columns, df.index

## IO

Velká výhoda `pandas` je, že umí číst souobry řady formátů pod sjednoceným interfacem, jehož použití je přímočaré. Výsledkem je typicky `pandas.Dataframe`. Např. soubor formátu `csv` načteme pomocí funkce `pandas.read_csv()`. Plný výčet podporovaných formátu nalezneme v dokumentaci: https://pandas.pydata.org/docs/user_guide/io.html .

## Čištění dat

Načtená data je často třeba protřídit, zbavit nežádoucích hodnot, přejmenovat sloupce apod. Mezi nejčastější takové operace patří následující:

### zahození, nahrazení nebo interpolace chybějících dat (NaN)

```python
df.dropna(inplace=True)
df['column_name'].fillna(0, inplace=True)
df['column_name'].interpolate(inplace=True)
```

### odstranění duplicit

```python
df.drop_duplicates(inplace=True)
```

### změna typu sloupce

```python
df['column_name'] = df['column_name'].astype('int')
```

### přejmenování sloupců

```python
df.rename(columns={'old_name': 'new_name'}, inplace=True)
```

### zahození sloupců

```python
df.drop(columns=['unwanted_column1', 'unwanted_column2'], inplace=True)
```

### datetime

```python
df['date_column'] = pd.to_datetime(df['date_column'])
```

### Filtrování a agregace

```python
mask1 = df['Age'] > 30
mask2 = df['Salary'] > 50000
filtered_df = df[mask1 & mask2]
```

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')
df.head()

In [ ]:
mask = df['Survived'] == 0
res = df[mask].groupby("Sex").agg({"Age": ["mean", "median"]})
res

In [ ]:
mask = df['Survived'] == 1
res = df[mask].groupby("Sex").agg({"Age": "mean"})
res

In [ ]:
df[df["Age"] < 15].count()

In [ ]:
df["Fare"].agg(["mean", "sum", "median"])

In [ ]:
df.hist("Age", bins=8, rwidth=0.8)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(df["Age"], bins=8, rwidth=0.8)

## Covid data

https://onemocneni-aktualne.mzcr.cz/covid-19

https://onemocneni-aktualne.mzcr.cz/api/v2/covid-19/nakazeni-vyleceni-umrti-testy.csv

In [ ]:
import pandas as pd

df = pd.read_csv("../../data/nakazeni-vyleceni-umrti-testy.csv")

df.set_index(["datum"], drop = True, inplace = True)
df.index = pd.to_datetime(df.index)

new_names = ["inf", "cur", "dead", "tests", "ag", "d_inf", "d_cur", "d_dead", "d_tests", "d_ag"]
df.columns = new_names

df["d_inf"].plot(color="red", ls=":")
df.resample("7D").mean()["d_inf"].plot(color = "yellow", lw = 4.0)


df2 = df.resample("7D").agg({"d_inf" : "mean", "inf" : "sum"})

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

fig, axs = plt.subplots(1, 2)

df["inf"].plot(ax=axs[0])
df["dead"].plot(ax=axs[1])